## Downloading Datasets
- Do not edit this cell


In [ ]:
import gdown
log_file_id = '12qD0Dt0-CFD9sniqxDBCKI5WCNb93FsQ'
sales_file_id = '1dPbMBAeJywAn0Lw0SdSRGAvA0Y_ePPOw'
apache_log = 'apache_log.txt'
sales = 'sales.csv'

gdown.download(f'https://drive.google.com/uc?export=download&id={log_file_id}', apache_log, quiet=False)
gdown.download(f'https://drive.google.com/uc?export=download&id={sales_file_id}', sales, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=12qD0Dt0-CFD9sniqxDBCKI5WCNb93FsQ
To: /content/apache_log.txt
100%|██████████| 63.6M/63.6M [00:01<00:00, 43.8MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1dPbMBAeJywAn0Lw0SdSRGAvA0Y_ePPOw
To: /content/sales.csv
100%|██████████| 45.9k/45.9k [00:00<00:00, 58.6MB/s]


'sales.csv'

## Parsing and loading as dataframe


In [ ]:
import pandas as pd
import re
from datetime import datetime
from datetime import time, date


# Define a function to parse a single log entry
def parse_log_entry(entry):
    log_pattern = (
        r"(?P<ip>[\d.]+) "  # IP address
        r"(?P<identity>\S+) "  # Identity
        r"(?P<userid>\S+) "  # User ID
        r"\[(?P<datetime>[^\]]+)\] "  # Date and time
        r'"(?P<method>\S+) '  # Method
        r"(?P<url>\S+) "  # URL
        r'(?P<protocol>\S+)" '  # Protocol
        r"(?P<status>\d+) "  # Status code
        r"(?P<size>\S+) "  # Size
        r'"(?P<referrer>[^\"]+)" '  # Referrer
        r'"(?P<user_agent>[^\"]+)" '  # User agent
        r"(?P<host>\S+) "  # Host
        r"(?P<dest_ip>[\d.]+)"  # Destination IP
    )

    match = re.match(log_pattern, entry)
    if match:
        data = match.groupdict()
        # Convert size to integer if possible
        data["size"] = int(data["size"]) if data["size"].isdigit() else 0
        # Convert datetime to a datetime object
        data["datetime"] = datetime.strptime(data["datetime"], "%d/%b/%Y:%H:%M:%S %z")
        return data
    else:
        return None


# Read the log file
log_file = apache_log
log_entries = []

with open(log_file, "r") as file:
    for line in file:
        parsed_entry = parse_log_entry(line.strip())
        if parsed_entry:
            log_entries.append(parsed_entry)

# Create a DataFrame
df = pd.DataFrame(log_entries)
df["day"] = df["datetime"].dt.day_name()
df["time"] = df["datetime"].dt.time
df["hour"] = df["datetime"].dt.hour
df["date"] = df["datetime"].dt.date
df["status"] = df["status"].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258034 entries, 0 to 258033
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype                    
---  ------      --------------   -----                    
 0   ip          258034 non-null  object                   
 1   identity    258034 non-null  object                   
 2   userid      258034 non-null  object                   
 3   datetime    258034 non-null  datetime64[ns, UTC-05:00]
 4   method      258034 non-null  object                   
 5   url         258034 non-null  object                   
 6   protocol    258034 non-null  object                   
 7   status      258034 non-null  int64                    
 8   size        258034 non-null  int64                    
 9   referrer    258034 non-null  object                   
 10  user_agent  258034 non-null  object                   
 11  host        258034 non-null  object                   
 12  dest_ip     258034 non-null  object         

### Q1

In [ ]:
start_time = time(12, 0, 0) ## Change this time according to the time given in your question
end_time = time(22, 0, 0) ## Change this time according to the time given in your question
url = "/hindi/" ## Change this url according to the url given in your question
day = "Monday" ## Change this day according to the day given in your question

condition = (
    (df["status"] >= 200)
    & (df["status"] < 300)
    & (df["method"] == "GET")
    & (df["url"].str.startswith(pat=url))
    & (df["time"] >= start_time)
    & (df["time"] <= end_time)
    & (df["day"] == day)
)
df[condition].shape[0]

186

### Q2

In [ ]:
start_time = time(3, 0, 0) ## Change this time according to the time given in your question
end_time = time(8, 0, 0) ## Change this time according to the time given in your question
url = "/hindi/" ## Change this url according to the url given in your question
day = "Wednesday" ## Change this day according to the day given in your question

condition = (
    (df["status"] >= 200)
    & (df["status"] < 300)
    & (df["method"] == "GET")
    & (df["url"].str.startswith(pat=url))
    & (df["time"] >= start_time)
    & (df["time"] <= end_time)
    & (df["day"] == day)
)
df[condition].shape[0]

173

###Q3

In [ ]:
url = "/blog/" ## Change this url according to the url given in your question
day = "Friday" ## Change this day according to the day given in your question

condition = (
    (df["url"].str.startswith(pat=url))
    & (df["day"] == day)
)
temp = df[condition].copy()
temp.groupby("hour")["ip"].nunique().max()

71

###Q4

In [ ]:
url = "/hindimp3/" ## Change this url according to the url given in your question
day = "Wednesday" ## Change this day according to the day given in your question

condition = (df["url"].str.startswith(pat=url)) & (df["day"] == day)
temp = df[condition].copy()
temp.groupby("hour")["ip"].nunique().max()

39

###Q5

In [ ]:
filter_date = date(2024, 5, 12) ## Change this date according to the date given in your question
url = "/malayalammp3/" ## Change this url according to the url given in your question

condition = (df["url"].str.startswith(pat=url)) & (df["date"] == filter_date)
temp = df[condition].copy()

# Group by hour and count requests
hourly_requests = temp.groupby("ip")["size"].sum()
hourly_requests.max()

2861

###Q6
### NOTE - This Question might be different Plz be carefull

In [ ]:
filter_date = date(2024, 5, 31) ## Change this date according to the date given in your question

condition = (df["date"] == filter_date) & (
    df["user_agent"].str.contains(pat="Chrome")
)
temp = df[condition].copy()

# Function to extract the major version of Chrome
def extract_chrome_version(user_agent):
    match = re.search(r"Chrome/(\d+)", user_agent)
    if match:
        return int(match.group(1))
    return None
temp["chrome_version"] = temp["user_agent"].apply(extract_chrome_version)
temp["chrome_version"].value_counts().max()

279

###Q7


In [ ]:
filter_date = date(2024, 5, 26) ## Change this date according to the date given in your question

condition = (df["date"] == filter_date) & (
    df["user_agent"].str.contains(pat="Chrome")
)
temp = df[condition].copy()

# Function to extract the major version of Chrome
def extract_chrome_version(user_agent):
    match = re.search(r"Chrome/(\d+)", user_agent)
    if match:
        return int(match.group(1))
    return None
temp["chrome_version"] = temp["user_agent"].apply(extract_chrome_version)
temp["chrome_version"].value_counts().max()

213

###Q8

In [ ]:
filter_date = date(2024, 5, 31) ## Change this date according to the date given in your question
condition = (df["date"] == filter_date) & (df["user_agent"].str.contains(pat="Chrome"))
temp = df[condition].copy()
temp["chrome_version"] = temp["user_agent"].apply(extract_chrome_version)
temp["chrome_version"].value_counts().max()

279

###Q9

In [ ]:
sales_df = pd.read_csv(sales)
sales_df.columns = ['city', 'product', 'sales']
product = "Juice" ## Change this product according to the product given in your question
sales_count = 24 ## Change this sales according to the sales given in your question

grapes_df = sales_df[(sales_df["product"] == product) & (sales_df["sales"] >= sales_count)]
# Group by normalized city and sum the sales
grouped_df = grapes_df.groupby("city")["sales"].sum().reset_index()
grouped_df["sales"].max()

750

###Q10

In [ ]:
sales_df = pd.read_csv(sales)
sales_df.columns = ['city', 'product', 'sales']
product = "Juice" ## Change this product according to the product given in your question
sales_count = 48 ## Change this sales according to the sales given in your question

hat_df = sales_df[(sales_df["product"] == product) & (sales_df["sales"] >= sales_count)]
grouped_df = hat_df.groupby("city")["sales"].sum().reset_index()
grouped_df["sales"].max()

644